## Parsing Courses on edx.org

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import traceback

In [2]:
#subjects

#subjects = ['Architecture','Art & Culture', 'Biology & Life Sciences', 'Business & Management', 'Chemistry', 'Communication', 'Computer Science', 'Data Analysis & Statistics', 'Design', 'Economics & Finance', 'Education & Teacher Training', 'Electronics', 'Energy & Earth Sciences', 'Engineering', 'Environmental Studies', 'Ethics', 'Food & Nutrition', 'Health & Safety', 'History', 'Humanities', 'Language', 'Law', 'Literature', 'Math', 'Medicine', 'Music', 'Philanthropy', 'Philosophy & Ethics', 'Physics', 'Science', 'Social Sciences']
subjects = ['Architecture']

#course tab on edx.org
base_website = 'https://www.edx.org/search?tab=course'

#query string as per subject
# eg for Chemistry, https://www.edx.org/search?tab=course&subject=Chemistry
website = 'https://www.edx.org/search?tab=course&subject={courseName}'

# maximum number of courses to parse per subject
MAX_COURSES_PER_SUBJECT = 27

In [3]:
# return url for the passed on subject
def subjectUrl(subName):
    subName = subName.replace(" ","%20").replace("&","%26")
    return website.format(courseName=subName)

In [4]:
## NOTE: Configure WebDriver and append webdriver's location into PATH system variable

# driver = webdriver.Chrome()
driver = webdriver.Edge()

### Data 
Description of the data frame fields

| subject |    Subject of course |
|---------|----------------------|
| title |      Course Title |
| desc |       Course Description|
|weeks|      Estimated Weeks required to complete|
|hours|      Average hours per week for completion|
|outcome|    What you'll learn|
|instit|         Hosting Institure|
|level|      Difficulty level of course|
|prereq|     Prerequisite of course|
|lang|       Language|
|url|        URL of course|



In [5]:
subj = []
title = []
desc = []
weeks = []
hours = []
instit = []
level = []
prereq = []
outcome = []
lang = []
url = []

In [6]:
def isPresent(xpth):
    lst = driver.find_elements( By.XPATH ,xpth)
    if ( len(lst) > 0):
        return lst[0].text
    else:
        return False 

try:
    for subject in subjects:

        #load the subject page
        driver.get(subjectUrl(subject))
        time.sleep(3)

        #Counter cnt
        cnt = 0

        #list of webelements with course links
        course_list_raw = []
        course_list = []

        #do ... while cnt <= MAX_COURSES_PER_SUBJECT and next is enabled
        while(cnt <= MAX_COURSES_PER_SUBJECT):

            iter_list = driver.find_elements(By.XPATH, "/html//main[@id='main-content']/div[@class='new-search-page search-results']/div[3]/div/div[@class='pgn__data-table-layout-wrapper']//div[@class='row']/div[*]/div[@role='group']/a")

            if ( MAX_COURSES_PER_SUBJECT > cnt + len(iter_list) ):
                #Filter out the urls from elements 
                course_list += list(map(lambda x: str(x.get_attribute('href')), iter_list))
                cnt += len(iter_list)

            else:
                #Filter out the urls from elements 
                course_list += list(map(lambda x: str(x.get_attribute('href')), iter_list[:(MAX_COURSES_PER_SUBJECT-cnt)]))
                cnt += MAX_COURSES_PER_SUBJECT-cnt
            
            # NEXT button on course result page
            NEXT_BTN = driver.find_element(By.XPATH,"/html//main[@id='main-content']/div[@class='new-search-page search-results']/div[3]/div//div[@class='pgn__data-table-wrapper']/div[1]/nav/ul[@class='pagination']//button[@class='btn next page-link']")
            
            # if cnt<MAX_COURSES_PER_SUBJECT and NEXT_BTN is enabled
            if(cnt<MAX_COURSES_PER_SUBJECT and NEXT_BTN.is_enabled()):
                NEXT_BTN.click()
                time.sleep(2)
            else:
                break

        # Subject and count
        print("\nSubject: ", subject)
        print("Count: ",cnt)
        #print(course_list)

        
        for course_page in course_list:

            driver.get(course_page)

            #tit = driver.find_element_by_xpath("/html//main[@id='main-content']/div[@class='course-about course-info-content']/div[@class='header']//h1").text
            tit = isPresent("/html//main[@id='main-content']/div[@class='course-about course-info-content']/div[@class='header']//h1")
            
            des = isPresent("//div[@class='p']")

            wee = isPresent("/html//main[@id='main-content']/div[@class='course-about course-info-content']/div[@class='d-flex flex-column flex-sm-column-reverse']/div[@class='course-snapshot-background']//div[@class='course-snapshot-content py-2 text-primary-500']/div/div[1]/div[@class='ml-3']/div[@class='h4 mb-0']")
            
            hou = isPresent("/html//main[@id='main-content']/div[@class='course-about course-info-content']/div[@class='d-flex flex-column flex-sm-column-reverse']/div[@class='course-snapshot-background']//div[@class='course-snapshot-content py-2 text-primary-500']/div/div[1]/div[@class='ml-3']/div[@class='small']")
            
            ins = isPresent("/html//main[@id='main-content']/div[@class='course-about course-info-content']/div[3]//div[@class='row']/div[1]/ul/li[1]")
            
            lev = isPresent("/html//main[@id='main-content']/div[@class='course-about course-info-content']/div[3]//div[@class='row']/div[1]/ul/li[3]")
            
            pre = isPresent("/html//main[@id='main-content']/div[@class='course-about course-info-content']/div[3]/div/div[2]//div[@class='row']/div[1]/ul//*[self::p or self::div]")

            out = driver.find_element(By.XPATH, "/html//main[@id='main-content']/div[@class='course-about course-info-content']/div[3]/div/div[3 or 4]/div[@class='preview-expand-component']/div[2]/div/*").get_attribute('innerHTML')
            
            lan = isPresent("/html//main[@id='main-content']/div[@class='course-about course-info-content']/div[3]//div[@class='row']/div[2]/ul/li[1]")
            
            #print(tit, des, wee, hou, ins, lev, pre, out, lan, sep='\n')
            if( not(tit and des and wee and hou and ins and lev and pre and lan and len(out)) ):
                continue
            
            # cleaning
            wee = wee.replace('Estimated ','').replace(' weeks', '')
            hou = hou.replace('hours per week','')
            ins = ins.replace('Institution: ', '')
            lev = lev.replace('Level: ', '')
            lan = lan.replace('Language: ','')

            # append values into list
            subj.append(subject)
            title.append(tit)
            desc.append(des)
            weeks.append(wee)
            hours.append(hou)
            instit.append(ins)
            level.append(lev)
            prereq.append(pre)
            outcome.append(out)
            lang.append(lan)
            url.append(course_page)
            
except Exception:
    print('ERROR: Process interupted due to an error at url', driver.current_url)
    print(traceback.format_exc())
    




Subject:  Architecture
Count:  27


In [7]:
#closing the browser
driver.close()

In [8]:
df = pd.DataFrame({'subject': subj, 'title': title, 'description': desc, 'weeks': weeks, 'hours': hours, 'institute': instit, 'level':level, 'prerequisites': prereq, 'outcomes': outcome, 'Languages': lang, 'url': url})
df.to_csv('edx-course.csv')
df

,subject,title,description,weeks,hours,institute,level,prerequisites,outcomes,Languages,url
0,Architecture,The Architectural Imagination,Learn fundamental principles of architecture —...,10,3–5,HarvardX,Introductory,None,"\n<li>How to read, analyze, and understand dif...",English,https://www.edx.org/course/the-architectural-i...
1,Architecture,Sustainability in Architecture: An Interdiscip...,This course introduces the basic elements and ...,4,3–4,UPValenciaX,Intermediate,"Basic engineering concepts on energy, carbon f...",\n<li>How to evaluate the sustainability perfo...,English,https://www.edx.org/course/sustainability-in-a...
2,Architecture,Site Planning Online,"How to plan a site, its infrastructure and pub...",10,4–6,MITx,Intermediate,"No prerequisities, but basic understanding of ...",How to analyze a site and imagine the possibil...,English,https://www.edx.org/course/site-planning-onlin...
3,Architecture,Introducción a los encofrados y las cimbras en...,Aprende sobre la construcción de obras civiles...,4,2–3,UPValenciaX,Introductory,No se requiere ningún prerrequisito previo,\n<li>Comprender la utilidad y las limitacione...,Español,https://www.edx.org/course/introduccion-a-los-...
4,Architecture,Japanese Architecture and Structural Design,"In this revised course, fundamental and modern...",5,1–3,TokyoTechX,Intermediate,"Basic knowledge of structural mechanics, dynam...",\n<li>Evolution of seismic design concepts in ...,English,https://www.edx.org/course/japanese-architectu...
5,Architecture,Sustainable Cities,"Learn how government, the private sector, and ...",9,4–6,SDGAcademyX,Intermediate,None.,"\n<li>An overview of governance, land manageme...",English,https://www.edx.org/course/sustainable-cities-...
6,Architecture,Sustainable Urban Development,Learn why cities are key in resolving global u...,7,4–6,Institutions: DelftXWageningenX,Introductory,There are no prerequisites for this course.,\n<li>The most relevant challenges that metrop...,English,https://www.edx.org/course/sustainable-urban-d...
7,Architecture,Circular Economy for a Sustainable Built Envir...,Learn how the principles of the Circular Econo...,6,3–5,DelftX,Intermediate,The course will be of interest and value to st...,At the end of the course you will be able to:,English,https://www.edx.org/course/circular-economy-fo...
8,Architecture,Ecodesign for Cities and Suburbs,"Using real examples, learn how ecology can gui...",6,4–5,UBCx,Introductory,None,<li>The principles of ecodesign and why it is ...,English,https://www.edx.org/course/ecodesign-for-citie...
9,Architecture,Energy Demand in Buildings,Discover how building design and occupancy det...,5,4–6,DelftX,Introductory,None,"\n<li>All about the energy chain approach, whi...",English,https://www.edx.org/course/energy-needs-in-bui...
